In [1]:
# sample using an Estimator model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np

# load mnist dataset
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

# Create the Estimator
def make_classifier(model_dir=None):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    
    def cnn_model_fn(features, labels, mode):
      import tensorflow.compat.v1 as tf
      tf.disable_v2_behavior()
    
      import numpy as np
      """Model function for CNN."""
      # Input Layer
      if isinstance(features, dict):
        features = features['x']
      input_layer = tf.reshape(features, [-1, 28, 28, 1])
        

      # Convolutional Layer #1
      conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu)

      # Pooling Layer #1
      pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

      # Convolutional Layer #2 and Pooling Layer #2
      conv2 = tf.layers.conv2d(
          inputs=pool1,
          filters=64,
          kernel_size=[5, 5],
          padding="same",
          activation=tf.nn.relu)
      pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

      # Dense Layer
      pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
      dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
      dropout = tf.layers.dropout(
          inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

      # Logits Layer
      logits = tf.layers.dense(inputs=dropout, units=10)

      predictions = {
          # Generate predictions (for PREDICT and EVAL mode)
          "classes": tf.argmax(input=logits, axis=1),
          # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
          # `logging_hook`.
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }

      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=model_dir)
    return mnist_classifier
    
mnist_classifier = make_classifier(model_dir="/tmp/mnist_convnet_model")
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])
# evaluate
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

2023-11-02 01:35:55.647364: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-02 01:35:55.648889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 01:35:55.668680: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 01:35:55.668701: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 01:35:55.668714: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, 

/tmp/ipykernel_372128/1736776514.py:35: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1 = tf.layers.conv2d(
/tmp/ipykernel_372128/1736776514.py:43: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
/tmp/ipykernel_372128/1736776514.py:46: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv2 = tf.layers.conv2d(
/tmp/ipykernel_372128/1736776514.py:52: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
/tmp/ipykernel_372128/1736776514.py:56: UserWarning: `tf.layers.dense` is de

INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1...
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:probabilities = [[0.10017282 0.09569    0.11281838 0.10454207 0.09254646 0.07568298
  0.08779176 0.09557801 0.11622369 0.11895383]
 [0.09421608 0.07744012 0.1103489  0.10977615 0.08261666 0.08294503
  0.10090858 0.11416768 0.12213971 0.10544109]
 [0.0992114  0.08789022 0.09678107 0.10990681 0.10090436 0.08950637
  0.09072031 0.10321424 0.11297487 0.10889037]
 [0.0937236  0.08610134 0.10815279 0.10808839 0.09534946 0.09660148
  0.0988886  0.09897055 0.09444247 0.11968125]
 [0.09906912 0.08833797 0.12070209 0.10984048 0.09892864 0.09685462
  0.09549295 0.09271861 0.09727667 0.10077877]
 [0.10090563 0.08179738 0.11260463 0.09166076 0.08012803 0.0944057
  0.08589818 0.11390393 0.12651286 0.11218295]
 [0.09069768 0.08601109 0.11095282 0.12072533 0.10012206 0

In [2]:
!ls /tmp/mnist_convnet_model

checkpoint			      model.ckpt-0.meta
eval				      model.ckpt-1.data-00000-of-00001
events.out.tfevents.1698885310.varda  model.ckpt-1.index
events.out.tfevents.1698885357.varda  model.ckpt-1.meta
graph.pbtxt			      model.ckpt-2.data-00000-of-00001
model.ckpt-0.data-00000-of-00001      model.ckpt-2.index
model.ckpt-0.index		      model.ckpt-2.meta


In [3]:
!pip freeze | grep dill

dill==0.3.5.1


In [4]:
import os
os.environ['OMEGA_FRAMEWORKS'] = "scikit-learn,tensorflow,keras"

import omegaml as om 

om.models.get('tf-model-mnist-fn')

/home/patrick/mambaforge/envs/omegaml-ce/lib/python3.10/site-packages/celery/fixups/django.py:56: FixupWarning: Environment variable DJANGO_SETTINGS_MODULE is defined
but Django isn't installed.  Won't apply Django fix-ups!

  warnings.warn(FixupWarning(ERR_NOT_INSTALLED))


In [5]:
# we can save the model so it can be reused and re-fitted on the cluster

import omegaml as om 
from omegaml.backends.tensorflow import TFEstimatorModel

om.datasets.put(train_data, 'mnist-X')
om.datasets.put(train_labels, 'mnist-Y')

saveable_model = TFEstimatorModel(estimator_fn=make_classifier, model=mnist_classifier, v1_compat=True)
om.models.put(saveable_model, 'tf-model-mnist-fn')


<Metadata: Metadata(name=tf-model-mnist-fn,bucket=omegaml,prefix=models/,kind=tfestimator.model,created=2023-11-02 01:36:02.204794)>

In [6]:
# fit on the cluster. this may take a while
om.runtime.model('tf-model-mnist-fn').fit('mnist-X', 'mnist-Y').get()

'<Metadata: Metadata(name=tf-model-mnist-fn,bucket=omegaml,prefix=models/,kind=tfestimator.model,created=2023-11-02 01:36:02.204000)>'

In [7]:
# re-load the model and predict from a numpy array
train_data = om.datasets.get('mnist-X')

X = train_data[0:2, :]

model_ = om.models.get('tf-model-mnist-fn')
[v for v in model_.predict(X)]

/tmp/ipykernel_372128/1736776514.py:46: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv2 = tf.layers.conv2d(
/tmp/ipykernel_372128/1736776514.py:47: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  inputs=pool1,
/tmp/ipykernel_372128/1736776514.py:55: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
/tmp/ipykernel_372128/1736776514.py:56: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
/tmp/ipykernel_372128/1736776514.py:58: UserWarning: `tf.layers.dense` is deprecated and will be removed in a f

[{'classes': 3,
  'probabilities': array([0.10634766, 0.08097006, 0.09523314, 0.12361997, 0.08386061,
         0.10128708, 0.09661846, 0.09962774, 0.1153264 , 0.097109  ],
        dtype=float32)},
 {'classes': 0,
  'probabilities': array([0.13886572, 0.06845843, 0.10557043, 0.10487789, 0.08427102,
         0.09124744, 0.10032602, 0.09087245, 0.11948048, 0.09603014],
        dtype=float32)}]

In [8]:
# predict on the cluster via direct input as numpy, or the numpy array stored as a dataset
train_data = om.datasets.get('mnist-X')

#we can save the data as stacked images
om.datasets.put(train_data[0:2, :], 'mnist-sample')
om.runtime.model('tf-model-mnist-fn').predict('mnist-sample').get()
# or just pass the data as is
om.runtime.model('tf-model-mnist-fn').predict(train_data[0:2, :]).get()

,classes,probabilities
0,3,"[0.10634766, 0.080970064, 0.09523314, 0.123619..."
1,0,"[0.13886572, 0.06845843, 0.10557043, 0.1048778..."


In [16]:
# or use the model REST API 
import requests
from omegaml.client.auth import AuthenticationEnv

url = getattr(om.defaults, 'OMEGA_RESTAPI_URL', 'http://localhost:5000')
auth = AuthenticationEnv.get_restapi_auth(om=om)
dataset = 'mnist-sample'
modelname = 'tf-model-mnist-fn'
predict_url = '{url}/api/v1/model/{modelname}/predict?datax={dataset}'.format(**locals())
resp = requests.put(predict_url, auth=auth, json={})
print(predict_url)
print(resp.json())

http://localhost:5000/api/v1/model/tf-model-mnist-fn/predict?datax=mnist-sample
{'model': 'tf-model-mnist-fn', 'result': [{'classes': 3, 'probabilities': [0.10634765774011612, 0.08097006380558014, 0.09523314237594604, 0.12361997365951538, 0.0838606134057045, 0.10128708183765411, 0.09661845862865448, 0.09962774068117142, 0.1153264045715332, 0.09710900485515594]}, {'classes': 0, 'probabilities': [0.13886572420597076, 0.0684584304690361, 0.10557042807340622, 0.10487788915634155, 0.08427102118730545, 0.09124743938446045, 0.10032601654529572, 0.09087245166301727, 0.11948047578334808, 0.09603013843297958]}], 'resource_uri': 'tf-model-mnist-fn'}
